Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploying a ML module on IoT Edge Device


In this exercise, we introduce the steps of deploying an ML module through [Azure IoT Edge](https://docs.microsoft.com/en-us/azure/iot-edge/how-iot-edge-works). The purpose is to deploy a trained machine learning model to the edge device. When the input data is generated from a particular process pipeline and fed into the edge device, the deployed model is able to make predictions right on the edge device without accessing to the cloud. 


## Outline<a id="BackToTop"></a>
- [Prerequisite](#prerequisite)
- [Step 1: Deploy ML Module on IoT Edge Device](#step1)
- [Step 2: Test ML Module](#step2)

In [ ]:
import os
import pandas as pd
from utilities import text_to_json
import requests
import numpy as np
import json
import subprocess
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model
from azureml.core.workspace import Workspace
from azureml.core.conda_dependencies import CondaDependencies
from dotenv import set_key, get_key, find_dotenv

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Prerequisites <a id="Prerequisite"></a>

### 1. Satisfy the requirment specified in Sections `Prerequisites` and `Setup` in the repo's [README page](./README.md).


### 2. Build the trained ML Model into a docker image. 
    
You have two options to satisfy this requirment. The first option is to use a prebuilt image. The second option is to complete all the notebooks from [00_AML_Configuration.ipynb](./00_AML_Configuration.ipynb) through [04_Create_Image_Deploy_On_AKS.ipynb](./04_Create_Image_Deploy_On_AKS.ipynb) (in [04 notebook](./04_Create_Image_Deploy_On_AKS.ipynb), complete through the section `Test image locally` only).




### 3. Create an IoT Hub and Register an IoT Edge device.

Please follow the sections `Create an IoT hub` and `Register an IoT Edge device` in document [Deploy Azure IoT Edge on a simulated device in Linux or MacOS - preview](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux). When creating IoT hub, we assume you use the same resource group as the one created in [00_AML_Configuration.ipynb](./00_AML_Configuration.ipynb). After finishing these instructions, copy the value for **Connection string—primary key** from the IoT Edge Device page and save it as variable *device_connection_string*. You will need to this information to fullfill next step. Please fill values to variables *iothub_name*, *device_id*, *deployment_id*, and *device_connection_string* in the next cell.
    



Get workspace

Load existing workspace from the config file.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep="\n")

In [ ]:
#device_connection_string = <device_connection_string>
iothub_name = 'myiotnub' # iot hub name
device_id = 'mlaksdevice' # the name you give to the edge device
deployment_id = 'mlaksmodulesdk'   # the ML module name

In [ ]:
set_key(env_path, "iothub_name", iothub_name)
set_key(env_path, "device_id", device_id)
set_key(env_path, "deployment_id", deployment_id)

In [ ]:
resource_group = get_key(env_path, 'resource_group')
image_name = get_key(env_path, 'image_name')

In [ ]:
# install az-cli iot extension 
!sudo -i az extension add --name azure-cli-iot-ext

Create IoT Hub

The following code creates a free F1 hub in the resource group IoTEdgeResources. Replace {hub_name} with a unique name for your IoT hub.

In [ ]:
!az iot hub create --resource-group $resource_group --name $iothub_name --sku F1 

Register an IoT Edge device

In the Azure cloud shell, enter the following command to create a device name in `device_id` variable in your hub.

In [ ]:
!az iot hub device-identity create --hub-name $iothub_name --device-id $device_id --edge-enabled

In [ ]:
json_data = !az iot hub device-identity show-connection-string --device-id $device_id --hub-name $iothub_name
device_connection_string = json.loads(''.join([i for i in json_data if 'WARNING' not in i]))['cs']
device_connection_string

### 4. Provision and Configure IoT Edge Device.

In this tutorial, we use a Ubuntu Linux VM as the edge device. You can use the same Linux VM where you run [the current notebook](./04b_Deploy_ML_model_on_IOT_Edge.ipynb) (Or alternatively, you can use another Linux VM, e.g. Ubuntu server 16.04 LTS). The goal is to configure the VM so that it can run IoT Edge runtime and Docker. 

To finish the configuration, please follow [this doc](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-install-iot-edge-linux) to install iot edge runtime on this VM. You will need to use *device_connection_string* obtained from previous section.

Follow section `Disable process identification` of [this doc](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning) to configure the DSVM. (Not sure what is the objective of this step, or what is the consequence if not performing this step)   

## Step 1: Deploy the ML module - Instructions via Azure portal (Tested working) <a id="step1"></a>


Essentially, the objective is to deploy the ML container to the IoT Edge device.
- Finish steps 1-4 of `Deploy to your device steps` section in [this doc](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning)


1. On the Azure portal, navigate to your IoT hub.
2. Click *IoT Edge (preview)* and select your IoT Edge device.
3. Select *Set modules*.
4. Select *Add IoT Edge Module*.
5. In the Name field, enter a name, `yourmodulename`. 
6. In the Image field, enter your image location; for example `yanzamlwacrzcoyaxuv.azurecr.io/imgmlaks:1`.
    
    【Tips】: You can find the image location in the resource group where AML workspace is created. In this resource group, click resource `Container registry`. You can find relivant information in `Access Keys` and `Repositories` tabs. 
    
7. In the *Container Create Options* field, set the following configuration. You can change the HostPort Binding port number to your desired port number. 

    {
      "HostConfig": {
        "PortBindings": {
          "5001/tcp": [
            {
              "HostPort": "5001"
            }
          ]
         }
        }
       }

8.	Click *Save*.
9.	Back in the *Add Modules* step, click *Next*.
10.	In the *Specify Routes* step, leave the default setting unchanged.      
11.	Select Next.
12.	In the *Review Deployment* step, click *Submit*.
13.	Return to the device details page and click *Refresh*. You should see the new `yourmodulename` running.


## Step 1: Deploy the ML module - Instructions via Python SDK (Not working yet)
Instructions from [this doc](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where).

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value
print('ContainerURL:{}'.format(image.image_location))
print('Servername: {}'.format(reg_name))
print('Username: {}'.format(username))
print('Password: {}'.format(password))

In [ ]:
# get the image from workspace in case the 'image' object is not in the memory
image_name = get_key(env_path, 'image_name')
image = ws.images[image_name]

In [ ]:
# this file has issue - need to fix
!wget https://raw.githubusercontent.com/Azure/ai-toolkit-iot-edge/master/amliotedge/deploymodel

In [ ]:
!sudo chmod +x deploymodel

In [ ]:
ContainerRegistryName = reg_name
imageLocationURL = image.image_location
DeploymentID = deployment_id
IoTHubname = iothub_name
DeviceID = device_id

In [ ]:
username

In [ ]:
# blocking cell. Not working yet. The downloaded file deploymodel has issue - working on it
!sudo ./deploymodel $ContainerRegistryName $username $password $imageLocationURL $DeploymentID $IoTHubname $DeviceID

In [ ]:
DeploymentID = 'mlaksmodulesdk2'
print(iothub_name)
print(DeploymentID)
print(device_id)

In [ ]:
# by pass file deploymodel, manually created deployment_test.json file. The test does not work yet.
!az iot edge deployment create --deployment-id $DeploymentID --content deployment_test.json --hub-name $iothub_name --target-condition "deviceId='$device_id'" --priority 1

In [ ]:
!az iot edge deployment create --deployment-id $DeploymentID --content module_deploy.json --hub-name $iothub_name --target-condition "deviceId='$device_id'" --priority 1

You can check the logs of the ML module with the below.

In [ ]:
!docker logs -f $yourmodulename

## Step 2: Test ML Module <a id="step2"></a>
We now test the ML Module from iot Edge device.

In [ ]:
%%time
prediction = aks_service.run(input_data = jsontext)
print(prediction)

Let's try a few more duplicate questions and display their top 3 original matches. Let's first get the scoring URL and and API key for the web service.

**The port 5001 must be open from Azure portal**. How to open it automatically?

In [ ]:
# The port 5001 must be open from Azure portal
#url = 'http://localhost:5001/score'
url = 'http://13.92.137.234:5001/score'

In [ ]:
#scoring_url = aks_service.scoring_uri
scoring_url = url
#api_key = aks_service.get_keys()[0]

In [ ]:
# call the web service end point
headers = {'Content-Type':'application/json'}
response = requests.post(scoring_url, data=jsontext, headers=headers)
response

In [ ]:
prediction = json.loads(response.content.decode('ascii'))
prediction

In [ ]:
dupes_to_score = dupes_test.iloc[:5,4]

In [ ]:
results = [
    requests.post(scoring_url, data=text_to_json(text), headers=headers)
    for text in dupes_to_score
]

Let's print top 3 matches for each duplicate question.

In [ ]:
[eval(results[i].json())[0:3] for i in range(0, len(results))]

Next let's quickly check what the request response performance is for the deployed model on IoT edge device.

In [ ]:
text_data = list(map(text_to_json, dupes_to_score))  # Retrieve the text data

In [ ]:
timer_results = list()
for text in text_data:
    res=%timeit -r 1 -o -q requests.post(scoring_url, data=text, headers=headers)
    timer_results.append(res.best)

In [ ]:
timer_results

In [ ]:
print("Average time taken: {0:4.2f} ms".format(10 ** 3 * np.mean(timer_results)))

# Issue

- No smooth flow to configure edge device - any automatic method?
- The port 5001 must be open from Azure portal. How to open it automatically?
- install az-cli iot extension - must use `sudo -i` - no documentation
   
   !sudo -i az extension add --name azure-cli-iot-ext
